In [1]:
from bs4 import BeautifulSoup
import requests
import time, os
import pickle
import re
import io
import pandas as pd

## Scrape race URLs

In [5]:
# Open main results page, capture the html
URL = "https://www.crossresults.com/?n=results&map=0&region=all"
response = requests.get(URL)
print(response.status_code)
soup = BeautifulSoup(response.text,'lxml')

200


In [43]:
# race data get appeneded to each of 4 lists
race_path = []
race_name = []
race_date = []
race_region = []

# pull race path, name, date and region. 
# divs belong to 2 classes. Go through each for every month of races
for div in soup.find_all(class_="monthContent"):
    for tr in div.find_all(class_="resultsrow datarow1"):
        race_path.append(tr.find('a').get('href'))
        race_name.append(tr.find('a').text)
        race_date.append(tr.find('a').findNext().text)
        race_region.append(tr.find('a').findNext().findNext().text)
    for tr in div.find_all(class_="resultsrow datarow2"):
        race_path.append(tr.find('a').get('href'))
        race_name.append(tr.find('a').text)
        race_date.append(tr.find('a').findNext().text)
        race_region.append(tr.find('a').findNext().findNext().text)


In [51]:
# We only want races in the US
US_regions = ['California', 'Mid Atlantic', 'Mountain West', 
              'New England', 'New York/Ontario', 'North Central', 
              'Pacific Northwest', 'South Central', 'Southeast']

# These IDs are for US races
US_ids = []

# Find if region is in the US, save ID if so
for idx, region in enumerate(race_region):
    if region in US_regions:
        US_ids.append(idx)

# Update lists to only include US races
race_path = [race_path[i] for i in US_ids]
race_name = [race_name[i] for i in US_ids]
race_date = [race_date[i] for i in US_ids]
race_region = [race_region[i] for i in US_ids]

In [55]:
# Put the data into one object
races = [race_path, race_name, race_date, race_region]

In [56]:
# Pickle the race data
with open('races.pickle', 'wb') as to_write:
    pickle.dump(races, to_write)

## Scrape data from each race

In [ ]:
# Read in the race data
with open('races.pickle','rb') as read_file:
    races = pickle.load(read_file)

In [89]:
# Open each race page, capture the html
path = '/race/10159'
URL = 'https://www.crossresults.com' + path
response = requests.get(URL)
soup = BeautifulSoup(response.text,'lxml')

In [123]:
# Race info
main = soup.find("div", {"id": "resultstitle"}).text.split(' • ')

# Race Name
name = main[0]

# Race date
date = ' '.join(main[1].split())

# Race location
location = main[2].split('\r')[0].strip()

# Beers
beers = soup.find("div", {"class": "beerrating rating"}).text.split()[0]

# Moisture
moisture = soup.find("div", {"class": "moisturerating rating"}).text.split()[0]

# Accel
accel = soup.find("div", {"class": "accelrating rating"}).text.split()[0]

# Tech
tech = soup.find("div", {"class": "techrating rating"}).text.split()[0]

# Elevation
elevation = soup.find("div", {"class": "elevationrating rating"}).text.split()[0]

# Conditions
conditions = soup.find("div", {"id": "resultstitle"}).text.strip().split('\n')[-1].strip()

# Weather
weather = conditions.split(',')[0]

# Temperature
temperature = conditions.split(',')[1].strip().split()[0]

# Wind
wind = conditions.split(',')[2].strip().split()[1]

# extract script tag to get lat and lon
script = soup.find('article', {'id': 'content'}).find('script', {'type': 'text/javascript'})

# pull out the lat and lon
pattern = re.compile('GetMap\(\"(.*?)"')
lat_lon = re.findall(pattern, script.string)[0]

In [79]:
# grab the result path
result_path = soup.find("span", {"class": "downloadoptions"}).find_all('a')[0]['href']

In [81]:
# capture results html
URL = 'https://www.crossresults.com/' + result_path
response = requests.get(URL)
result_soup = BeautifulSoup(response.text,'lxml')

In [147]:
# read the results into a pandas dataframe
race_df = pd.read_csv(io.StringIO(result_soup.text.strip()), index_col=False)

In [148]:
race_df

,Category Name,Place,RacerID,First Name,Last Name,Team Name,Time,License,Carried Points,Scored Points
0,Men Cat 1/2,1,74538,Ryan,Woodall,Team TGB,NaN,153440.0,NaN,NaN
1,Men Cat 1/2,2,195292,Nick,Mackie,NaN,NaN,558841.0,NaN,NaN
2,Men Cat 1/2,3,136346,Matthew,Nalesnik,NaN,NaN,119092.0,NaN,NaN
3,Men Cat 3,1,195292,Nick,Mackie,NaN,NaN,558841.0,421.797987,366.539396
4,Men Cat 3,2,136346,Matthew,Nalesnik,NaN,NaN,119092.0,400.000000,377.692931
...,...,...,...,...,...,...,...,...,...,...
74,Women Junior 16-18,1,183163,Ashley,Davis,Velobrew Racing,NaN,476708.0,NaN,NaN
75,Women Junior 9-12,1,197020,Isabella,Fountain,NaN,NaN,NaN,NaN,NaN
76,Women Junior 9-12,2,197016,Eve,Munkittrick,NaN,NaN,590776.0,NaN,NaN
77,Women Junior 9-12,3,197021,Helen,Munkittrick,NaN,NaN,604405.0,NaN,NaN


In [150]:
race_df[['Race Name', 'Date', 'Location', 
         'Beers', 'Moisture', 'Accel', 
         'Tech', 'Elevation', 'Weather', 
         'Temperature', 'Wind', 'Coordinates']] = name, date, location, \
                                                  beers, moisture, accel, \
                                                  tech, elevation, weather, \
                                                  temperature, wind, lat_lon

race_df

,Category Name,Place,RacerID,First Name,Last Name,Team Name,Time,License,Carried Points,Scored Points,...,Location,Beers,Moisture,Accel,Tech,Elevation,Weather,Temperature,Wind,Coordinates
0,Men Cat 1/2,1,74538,Ryan,Woodall,Team TGB,NaN,153440.0,NaN,NaN,...,"Green Cove Springs, FL",2.2,3.6,2.4,2.2,2.4,Partly Cloudy,49,4,29.9884166717529:-81.6820068359375
1,Men Cat 1/2,2,195292,Nick,Mackie,NaN,NaN,558841.0,NaN,NaN,...,"Green Cove Springs, FL",2.2,3.6,2.4,2.2,2.4,Partly Cloudy,49,4,29.9884166717529:-81.6820068359375
2,Men Cat 1/2,3,136346,Matthew,Nalesnik,NaN,NaN,119092.0,NaN,NaN,...,"Green Cove Springs, FL",2.2,3.6,2.4,2.2,2.4,Partly Cloudy,49,4,29.9884166717529:-81.6820068359375
3,Men Cat 3,1,195292,Nick,Mackie,NaN,NaN,558841.0,421.797987,366.539396,...,"Green Cove Springs, FL",2.2,3.6,2.4,2.2,2.4,Partly Cloudy,49,4,29.9884166717529:-81.6820068359375
4,Men Cat 3,2,136346,Matthew,Nalesnik,NaN,NaN,119092.0,400.000000,377.692931,...,"Green Cove Springs, FL",2.2,3.6,2.4,2.2,2.4,Partly Cloudy,49,4,29.9884166717529:-81.6820068359375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,Women Junior 16-18,1,183163,Ashley,Davis,Velobrew Racing,NaN,476708.0,NaN,NaN,...,"Green Cove Springs, FL",2.2,3.6,2.4,2.2,2.4,Partly Cloudy,49,4,29.9884166717529:-81.6820068359375
75,Women Junior 9-12,1,197020,Isabella,Fountain,NaN,NaN,NaN,NaN,NaN,...,"Green Cove Springs, FL",2.2,3.6,2.4,2.2,2.4,Partly Cloudy,49,4,29.9884166717529:-81.6820068359375
76,Women Junior 9-12,2,197016,Eve,Munkittrick,NaN,NaN,590776.0,NaN,NaN,...,"Green Cove Springs, FL",2.2,3.6,2.4,2.2,2.4,Partly Cloudy,49,4,29.9884166717529:-81.6820068359375
77,Women Junior 9-12,3,197021,Helen,Munkittrick,NaN,NaN,604405.0,NaN,NaN,...,"Green Cove Springs, FL",2.2,3.6,2.4,2.2,2.4,Partly Cloudy,49,4,29.9884166717529:-81.6820068359375
